## Gemini BuildWithAI 2025 Workshop

<a target="_blank" href="https://colab.research.google.com/github/mashhoodr/gemini-cookbook/blob/main/workshops/gemini-vibecoding.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>


This notebook is designed to help you become familiar with Gemini's API while creating some simple projects. We use "vibe coding" to help build some basic apps and test out the power of Gemini.

### Learning Outcomes

The objective of this workshop is to help the attendees become familiar with the offerings of Google Gemini, and give them an opportunity to try out the API themselves. We run through a few exercises to help understand the use cases for the different functionalities present.

### Authentication

The Gemini API uses API keys for authentication. We will now setup the API key in this colab - and test out our authentication. Your trainer has already demoed the instructions below.

You can [create](https://aistudio.google.com/app/apikey) your API key using Google AI Studio with a single click.  

Remember to treat your API key like a password. Do not accidentally save it in a notebook or source file you later commit to GitHub. This notebook shows you two ways you can securely store your API key.

* If you are using Google Colab, we recommend you store your key in Colab Secrets.

* If you are using a different development environment (or calling the Gemini API through `cURL` in your terminal), we recommend you store your key in an environment variable.

Let's start with Colab Secrets.

Add your API key to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 **Secrets** tab in the left panel.
   
   <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>

2. Create a new secret with the name `GOOGLE_API_KEY`.
3. Copy/paste your API key into the `Value` input box of `GOOGLE_API_KEY`.
4. Toggle the button on the left to allow notebook access to the secret.


### Install the Python SDK

In [ ]:
!pip install -q -U google-genai

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Configure the SDK with your API key.

You'll call `genai.configure` with your API key, but instead of pasting your key into the notebook, you'll read it from Colab Secrets.

In [ ]:
from google import genai
from google.genai import types
from google.colab import userdata
from pydantic import BaseModel
from IPython.display import Markdown, HTML, Image

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

And that's it! Now you're ready to use the Gemini API.

Now lets set our model we want to use throughout this notebook. You can change this to any available model.

In [ ]:

MODEL = "gemini-2.0-flash"

### Running your first prompt

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to generate_content, and use the `.text` property to get the text content of the response.

In [ ]:
response = client.models.generate_content(
    model=MODEL, contents="Explain how AI works in a few words."
)

print(response.text)

> *Do it yourself: Update the above using a different Gemini version available. Did the response change?*

#### Use images in your prompt

Here we download an image from a URL and pass that image in our prompt.

First, we download the image and load it with PIL:

In [ ]:
!curl -o image.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

In [ ]:
import PIL.Image
img = PIL.Image.open('image.jpg')
img

In [ ]:
prompt = """This image contains a sketch of a potential product along with some notes.
Given the product sketch, describe the product as thoroughly as possible based on what you
see in the image, making sure to note all of the product features. Return output in json format:
{description: description, features: [feature1, feature2, feature3, etc]}"""

Then we can include the image in our prompt by just passing a list of items to `generate_content`. You can pass in multiple images, or prompts or files as per your requirement. 

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents=[prompt, img]
)
print(response.text)

#### Uploading files

For types other than image, like audio, video or pdf - you can use the `upload_file` function to send data to Gemini.

The following list of documents are supported:

- PDF - application/pdf
- JavaScript - application/x-javascript, text/javascript
- Python - application/x-python, text/x-python
- TXT - text/plain
- HTML - text/html
- CSS - text/css
- Markdown - text/md
- CSV - text/csv
- XML - text/xml
- RTF - text/rtf

First download a PDF file into Colab.

In [ ]:
URL = "https://storage.googleapis.com/generativeai-downloads/data/Smoothly%20editing%20material%20properties%20of%20objects%20with%20text-to-image%20models%20and%20synthetic%20data.pdf"
!wget -q $URL -O sample.pdf

Then pass it into Gemini.

In [ ]:
import pathlib
import io
import httpx


your_file = client.files.upload(file='sample.pdf')
prompt = "Can you summarize this file as a bulleted list?"

response = client.models.generate_content(
  model=MODEL,
  contents=[your_file, prompt])

Markdown(response.text)

#### Supply a schema through model configuration
The following example does the following:

Instantiates a model configured through a schema to respond with JSON.
Prompts the model to return cookie recipes.

The Gemini API Python client library supports schemas defined with the following types (where AllowedType is any allowed type):

- int
- float
- bool
- str
- list[AllowedType]

In [ ]:

class Recipe(BaseModel):
  recipe_name: str
  ingredients: list[str]

response = client.models.generate_content(
    model=MODEL,
    contents='List a few popular cookie recipes.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Recipe],
    },
)

# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed

#### Use an enum to constrain output
In some cases you might want the model to choose a single option from a list of options. To implement this behavior, you can pass an enum in your schema. You can use an enum option anywhere you could use a str in the response_schema, because an enum is a list of strings. Like a JSON schema, an enum lets you constrain model output to meet the requirements of your application.

For example, assume that you're developing an application to classify musical instruments into one of five categories: "Percussion", "String", "Woodwind", "Brass", or ""Keyboard"". You could create an enum to help with this task.

In [ ]:
import enum

class Instrument(enum.Enum):
  PERCUSSION = "Percussion"
  STRING = "String"
  WOODWIND = "Woodwind"
  BRASS = "Brass"
  KEYBOARD = "Keyboard"

response = client.models.generate_content(
    model=MODEL,
    contents='What type of instrument is an oboe?',
    config={
        'response_mime_type': 'text/x.enum',
        'response_schema': Instrument,
    },
)

print(response.text)

#### Have a chat

The Gemini API enables you to have freeform conversations across multiple turns.

The [ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) class will store the conversation history for multi-turn interactions.

In [ ]:
chat = client.chats.create(model=MODEL)
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
print(response.text)

You can see the chat history:

In [ ]:
for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

You can send another message to continue the conversation. The previous conversation is automatically sent in the next message as context.

In [ ]:
response = chat.send_message("What are the main components of a computer?")
print(response.text)

### Setting the system instruction

The system instruction in Gemini is a tool for developers to fine-tune the model's responses for specific tasks. It lets them define various aspects of how Gemini should generate responses [2].

Here are some key benefits of system instructions:

**Role definition:** You can specify the role Gemini should play, such as a home-cooking assistant or a music historian.

**Format control:** Instruct Gemini on the format of the response, like text, a list, or even a structured JSON object.

**Goal setting:** Clearly define the goal you want Gemini to achieve, making the response more focused and relevant.

**Rule establishment:** Set rules for Gemini to follow, ensuring the response adheres to your specific requirements.

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents=["Share a short story for children on kindness."],
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1,
        system_instruction="You are a primary school teacher specializing in early childhood education. Use positive reinforcement and interactive methods to teach basic concepts. Adapt your responses to the learning style of a young child."
    )
)
print(response.text)


## Let's do some practice!

In the world of AI Agents and Coding tools from LLMs, there is a new way to learn. Instead of reading API documentations and creating demos by hand, we want to use the power of LLMs to quickly build the scaffolding for us. This allows us to play with the technology at a much deeper level, rather than spending hours before just "setting up" some basic things.

For our workshop today, we will be using the recently launched **Firebase Studio**. Access it on https://firebase.studio/ and lets create the first workspace for the exercise below. 

If for any reason Firebase Studio isnt working, then I would recommend https://lovable.dev/

Alternatively those comfortable using Cursor, Copilot, Replit or Windsurf - feel free to use that. I would recommend using the Gemini Pro 2.5 API with them for a better Gemini SDK integration experience.

### Exercise 1: A Simple Story Generator with Gemini Chat API

**Problem statement**

When kids are learning how to read - for every kid its a very difficult and slow process. The only way to get better is practice. However the practice needs to be at the right level, simply reading the same book a thousand times isnt good enough. 

Enter our story generator. 
1. It can be designed to generate content at the right level, so for example grade one. 
2. It can generate new content every time
3. It can generate content which appeals to the child, where they are the hero (check link below).

The right product will help the kids learn to read better over time.

**Theme Selection**

Present a list of themes (e.g., fantasy, sci-fi, mystery, historical) when the session starts.
Allow the user to input a theme or select from the list.

**Initial Story Generation**

Based on the selected theme, generate a short paragraph introducing the story and the user's character. Ensure its safe for children, uses easy to use language and as creative as possible.

**Action Selection**

Provide multiple action choices related to the current story.
Allow the user to select an action.

**Story Continuation**

Generate a new paragraph based on the user's chosen action, advancing the story.
Repeat steps 3 and 4 until a desired story length or ending condition is reached.

Idea is inspired by https://www.wander.ly/ - check their website for more inspiration.

#### Let's continue to dive deeper with Function calling

To use function calling, pass a list of functions to the `tools` parameter when creating a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel). The model uses the function name, docstring, parameters, and parameter type annotations to decide if it needs the function to best answer a prompt.

> Important: The SDK converts function parameter type annotations to a format the API understands (`glm.FunctionDeclaration`). The API only supports a limited selection of parameter types, and the Python SDK's automatic conversion only supports a subset of that: `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`

In [ ]:
def add(a:float, b:float):
    """returns a + b."""
    return a+b

def subtract(a:float, b:float):
    """returns a - b."""
    return a-b

def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

def divide(a:float, b:float):
    """returns a / b."""
    return a*b


config = {
    "tools": [add, subtract, multiply, divide],
}
chat = client.chats.create(model=MODEL, config=config)
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

However, by examining the chat history, you can see the flow of the conversation and how function calls are integrated within it.

The `ChatSession.history` property stores a chronological record of the conversation between the user and the Gemini model. Each turn in the conversation is represented by a [`glm.Content`](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) object, which contains the following information:

*   **Role**: Identifies whether the content originated from the "user" or the "model".
*   **Parts**: A list of [`glm.Part`](https://ai.google.dev/api/python/google/ai/generativelanguage/Part) objects that represent individual components of the message. With a text-only model, these parts can be:
    *   **Text**: Plain text messages.
    *   **Function Call** ([`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)): A request from the model to execute a specific function with provided arguments.
    *   **Function Response** ([`glm.FunctionResponse`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)): The result returned by the user after executing the requested function.

 In the previous example with the mittens calculation, the history shows the following sequence:

1.  **User**: Asks the question about the total number of mittens.
1.  **Model**: Determines that the multiply function is helpful and sends a FunctionCall request to the user.
1.  **User**: The `ChatSession` automatically executes the function (due to `enable_automatic_function_calling` being set) and sends back a `FunctionResponse` with the calculated result.
1.  **Model**: Uses the function's output to formulate the final answer and presents it as a text response.

In [ ]:
for content in chat.get_history():
    print(content.role, "->", [(str(part.function_call) + ' -> ' + str(part.function_response)) for part in content.parts])
    print('-'*80)

#### Use Model Context Protocol (MCP)

Model Context Protocol (MCP) is an open standard to connect AI applications with external tools, data sources, and systems. MCP provides a common protocol for models to access context, such as functions (tools), data sources (resources), or predefined prompts. You can use models with MCP server using their tool calling capabilities.

MCP servers expose the tools as JSON schema definitions, which can be used with Gemini compatible function declarations. This lets you to use a MCP server with Gemini models directly. You can learn more about MCP and how to use it in the documentation: 

https://ai.google.dev/gemini-api/docs/function-calling?example=weather#use_model_context_protocol_mcp


### Using Code Execution 

The Gemini API code execution feature enables the model to generate and run Python code and learn iteratively from the results until it arrives at a final output. You can use this code execution capability to build applications that benefit from code-based reasoning and that produce text output. For example, you could use code execution in an application that solves equations or processes text.

The code execution environment includes the following libraries: altair, chess, cv2, matplotlib, mpmath, numpy, pandas, pdfminer, reportlab, seaborn, sklearn, statsmodels, striprtf, sympy, and tabulate. You can't install your own libraries.

In [ ]:
response = client.models.generate_content(
  model=MODEL,
  contents='What is the sum of the first 50 prime numbers? '
           'Generate and run code for the calculation, and make sure you get all 50.',
  config=types.GenerateContentConfig(
    tools=[types.Tool(
      code_execution=types.ToolCodeExecution
    )]
  )
)

def display_code_execution_result(response):
  for part in response.candidates[0].content.parts:
    if part.text is not None:
      display(Markdown(part.text))
    if part.executable_code is not None:
      code_html = f'<pre style="background-color: #BBBBEE;">{part.executable_code.code}</pre>' # Change code color
      display(HTML(code_html))
    if part.code_execution_result is not None:
      display(Markdown(part.code_execution_result.output))
    if part.inline_data is not None:
      display(Image(data=part.inline_data.data, format="png"))
    display(Markdown("---"))

display_code_execution_result(response)

#### Prompt Caching

One of the cool new features which has been added by Gemini is prompt caching. If a part of your prompt or instructions is not changing, you can save some tokens by caching that part of the prompt. This is very useful in production where the same prompt might be used for thousands and millions of times and allows us to optimize cost.

In order to use the cache, we create a cache context and then call the generate content endpoint with the additional context.

First we download some data which is reused again and again across context.

In [ ]:
!wget -q https://storage.googleapis.com/generativeai-downloads/data/a11.txt
!head a11.txt

In [ ]:
import datetime

# upload the files.
document = client.files.upload(file="a11.txt")

cache_model='models/gemini-2.0-flash-001'

# Create a cache with a 5 minute TTL
cache = client.caches.create(
    model=cache_model,
    config=types.CreateCachedContentConfig(
      display_name='transcript',
      system_instruction=(
          "You are an expert at analyzing transcripts."
      ),
      contents=[document],
      ttl="300s",
  )
)

# Construct a GenerativeModel which uses the created cache.
response = client.models.generate_content(
  model = cache_model,
  contents= ("Find a lighthearted moment from this transcript"),
  config=types.GenerateContentConfig(cached_content=cache.name)
)


print(response.text)


print(response.usage_metadata)

In [ ]:
# Once you have used the cache you can also delete it (or it expires automatically at the given time)
client.caches.delete(name=cache.name)

### Exercise 2: A Weather App using Function Calling.

Create a simple weather app using Firebase Studio which will use function calling to fetch the latest weather when asked about for a specific location. 

An example prompt for it can be: `Whats the weather like in Islamabad today?`

You can use the free api available on: https://www.weatherapi.com/

Bonus: Configure the temperature unit toggle as well (C or F), with C being default.

When you ask for `Hows the weather in Karachi this week?` is it able to adjust its UI to show the result?

What happens when you ask it a question like `Do I need an umbrella in Islamabad today?`

### Exercise 3: An offline-first Learning Management System (LMS)

This is a slightly more complex one, we definitely dont want to single shot this app. I have listed down some prompts/requirements below - you can use these to build up the app step by step. You can choose to change these prompts or add any other functionality you like as well. The focus here is on integrating Gemini to generate courses, lessons and assessments. 

As you build up the functionality, think about testing this app, and how you will ensure that nothing is breaking after each iteration.

1. "I want to build a personal learning management system, similar to Udemy where I can create courses and track my progress in them over time. It should all work offline on the client using localstorage."
2. It should contain a structure having courses, sections, lessons first. We will add assessments later.
3. You should be able to add, edit or delete courses, sections and lessons. They should persist on refreshing the page.
3. Ability to add Youtube links in Lessons.
4. Ability to manually add assessments (basic mcqs) in the Lessons.
5. Move everything to IndexedDB. (refactoring ability)
6. Feature to generate Sections and Lessons using Gemini.
7. Feature to generate assessments using Gemini.
8. Use the caching feature to upload a large PDF or video and use that to create multiple lessons and assessments.

## Thank you.

Thank you for attending this workshop. You can find more details about me on https://karachiwala.dev. I am available over most platforms as @mashhoodr.

You can find many more examples for Gemini on the following repositories.

- https://github.com/google-gemini/cookbook
- https://github.com/GoogleCloudPlatform/generative-ai

If you have any feedback on this workshop please share it with me using the following link:

https://docs.google.com/forms/d/1iAEO1JSlh6GTLC0uudUxAiTDiNN_iMzfdCwDLZ_78sg/edit
